### 2631 Assignment 2  - Modeling

### Yiming Gu, 
### Shenghua Yue

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import timedelta
import lightgbm as lgb

In [2]:
# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.options.display.max_columns = 999

In [5]:
#input the preprocessed data 
train = pd.read_csv('train_all.csv',low_memory=True)

In [6]:
test = pd.read_csv('test_all.csv',low_memory=True)

In [7]:
train.shape
test.shape

(1708337, 38)

(401589, 37)

In [8]:
train['date'] = pd.to_datetime(train.date.apply(lambda x: str(x)[0:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8]))
test['date'] = pd.to_datetime(test.date.apply(lambda x: str(x)[0:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8]))

In [9]:
train.drop(['campaignCode'], axis=1,inplace=True)

In [10]:
#combine the train and test data for splitting the time windows (which will be explained later)
all_data = pd.concat([train, test])

In [11]:
# check and handle the missing values
all_data.isna().sum()

Unnamed: 0                       0
adContent                  1643600
bounces                    1055670
browser                          0
campaign                         0
channelGrouping                  0
city                             0
continent                        0
country                          0
date                             0
deviceCategory                   0
fullVisitorId                    0
hits                             0
index                      1034549
isMobile                         0
isTrueDirect               1426999
keyword                    1093006
medium                           0
metro                            0
networkDomain                    0
newVisits                   516431
operatingSystem                  0
pageviews                      340
referralPath               1142073
region                           0
sessionQualityDim           835274
source                           0
subContinent                     0
timeOnSite          

In [12]:
all_data.transactionRevenue.fillna(0,inplace=True)
all_data.totalTransactionRevenue.fillna(0,inplace=True)
all_data.bounces.fillna(0,inplace=True)
all_data.isTrueDirect.fillna(False, inplace=True)
all_data.newVisits.fillna(0,inplace=True)
all_data.sessionQualityDim.fillna(0,inplace=True)
all_data.transactions.fillna(0,inplace=True)
all_data.pageviews.fillna(0,inplace=True)
all_data.timeOnSite.fillna(0,inplace=True)

In [13]:
all_data.dropna(thresh=1000000, axis=1, inplace=True)

In [14]:
dropcol = ['Unnamed: 0','index', 'keyword', 'value']
all_data.drop(columns = dropcol, inplace=True)

In [15]:
all_data.isna().sum().sum()
all_data.shape

0

(2109926, 31)

In order to simulate the prediction situation, where the data used for prediction is within 168 days and the 62-day time window to be predicted is 46 days later. Thus, the whole dataset is splitted to four 168-day time periods as the data for generating dependent variables. Correspondingly, four 62-day time windows after the 46 days gap are found and considered as response time windows from where the target value is calculated.

In [16]:
[train.date.min(), train.date.max(), train.date.max() - train.date.min()]
[test.date.min(),test.date.max(), test.date.max() - test.date.min()]
[all_data.date.min(),all_data.date.max(), all_data.date.max() - all_data.date.min()]

[Timestamp('2016-08-01 00:00:00'),
 Timestamp('2018-04-30 00:00:00'),
 Timedelta('637 days 00:00:00')]

[Timestamp('2018-05-01 00:00:00'),
 Timestamp('2018-10-15 00:00:00'),
 Timedelta('167 days 00:00:00')]

[Timestamp('2016-08-01 00:00:00'),
 Timestamp('2018-10-15 00:00:00'),
 Timedelta('805 days 00:00:00')]

In [17]:
for i in range(4):
    print ('Time period ' + str(i+1) + ': '+ str(all_data.date.min() + timedelta(days=168)*i)[0:10] + ' to ' + str(all_data.date.min() + timedelta(days=168)*i + timedelta(days=167))[0:10])
    print ('Response Window ' + str(i+1) + ': ' + str(all_data.date.min() + timedelta(days = 168)*(i+1) + timedelta(days = 46))[0:10] + ' to ' + str(all_data.date.min() + timedelta(days = 168)*(i+1) + timedelta(days = 108))[0:10])

Time period 1: 2016-08-01 to 2017-01-15
Response Window 1: 2017-03-03 to 2017-05-04
Time period 2: 2017-01-16 to 2017-07-02
Response Window 2: 2017-08-18 to 2017-10-19
Time period 3: 2017-07-03 to 2017-12-17
Response Window 3: 2018-02-02 to 2018-04-05
Time period 4: 2017-12-18 to 2018-06-03
Response Window 4: 2018-07-20 to 2018-09-20


In [18]:
# divide the data into 4 time periods
tp1 = all_data[all_data.date <= '2017-01-15']
tp2 = all_data[(all_data.date >= '2017-01-16') & (all_data.date <= '2017-07-02')]
tp3 = all_data[(all_data.date >= '2017-07-03') & (all_data.date <= '2017-12-17')]
tp4 = all_data[(all_data.date >= '2017-12-18') & (all_data.date <= '2018-06-03')]

In [19]:
# get the target values in the corresponding response windows
rw1 = all_data[(all_data.date >= '2017-03-03') & (all_data.date <= '2017-05-04')]\
[['fullVisitorId','transactionRevenue','date']]
rw2 = all_data[(all_data.date >= '2017-08-18') & (all_data.date <= '2017-10-19')]\
[['fullVisitorId','transactionRevenue','date']]
rw3 = all_data[(all_data.date >= '2018-02-02') & (all_data.date <= '2018-04-05')]\
[['fullVisitorId','transactionRevenue','date']]
rw4 = all_data[(all_data.date >= '2018-07-20') & (all_data.date <= '2018-09-20')]\
[['fullVisitorId','transactionRevenue','date']]

In [20]:
# Integrate the observations by per visitor
# TP1 process

In [21]:
groupSize1 = tp1.groupby('fullVisitorId').size().to_frame()
groupSize1.columns=['size']
tp1size = tp1.merge(groupSize1,how='left',left_on='fullVisitorId',right_index=True)
tp1_dup = tp1size[tp1size['size']>1]
tp1_uni = tp1size[tp1size['size']==1]

In [22]:
tp1_dup_s = tp1_dup.groupby('fullVisitorId').agg({'bounces':'sum',\
                                  'browser':lambda x:x.value_counts().index[0],\
                                  'campaign':lambda x:x.value_counts().index[0],\
                                  'channelGrouping':'max',\
                                  'city':lambda x:x.value_counts().index[0],\
                                  'continent':lambda x:x.value_counts().index[0],\
                                  'country':lambda x:x.value_counts().index[0],\
                                  'date':['count',lambda x: (x.max() - x.min()).days],\
                                  'deviceCategory':lambda x:x.value_counts().index[0],\
                                  'hits':['sum','min','max','mean','std'],\
                                  'isMobile':lambda x:x.value_counts().index[0],\
                                  'isTrueDirect':lambda x:x.value_counts().index[0],\
                                  'medium':lambda x:x.value_counts().index[0],\
                                  'metro':lambda x:x.value_counts().index[0],\
                                  'networkDomain':lambda x:x.value_counts().index[0],\
                                  'newVisits':'max',
                                  'operatingSystem':lambda x:x.value_counts().index[0],\
                                  'pageviews':['sum','min','max','mean','std'],\
                                  'region':lambda x:x.value_counts().index[0],\
                                  'source':lambda x:x.value_counts().index[0],\
                                  'subContinent':lambda x:x.value_counts().index[0],\
                                  'timeOnSite':'sum',\
                                  'transactionRevenue':'sum',\
                                  'transactions':'max',\
                                  'visitNumber':'max'})

In [23]:
tp1_dup_s.columns = ['bounces','browser','campaign','channelGrouping','city',\
                            'continent','country','dateCount','dateDiff','deviceCategory','hitsSum',\
                            'hitsMin','hitsMax','hitsMean','hitsStd','isMobile','isTrueDirect','medium',\
                            'metro','networkDomain','newVisits','operatingSystem','pageViewsSum','pageViewsMin',\
                           'pageViewsMax','pageViewsMean','pageViewsStd','region','source','subContinent',\
                            'timeOnSite','transactionRevenue','transactions','visitNumber']

In [24]:
tp1_uni_s = tp1_uni[['bounces','browser','campaign','channelGrouping','city',\
                    'continent','country','date','deviceCategory','hits',\
                    'isMobile','isTrueDirect','medium',\
                    'metro','networkDomain','newVisits','operatingSystem','pageviews',\
                    'region','source','subContinent',\
                    'timeOnSite','transactionRevenue','transactions','visitNumber']]

In [25]:
tp1_uni_s['dateCount'] = 1
tp1_uni_s['dateDiff'] = 0
tp1_uni_s['hitsSum'] = tp1_uni_s['hits']
tp1_uni_s['hitsMin'] = tp1_uni_s['hits']
tp1_uni_s['hitsMax'] = tp1_uni_s['hits']
tp1_uni_s['hitsMean'] = tp1_uni_s['hits']
tp1_uni_s['hitsStd'] = 0
tp1_uni_s['pageViewsSum'] = tp1_uni_s['pageviews']
tp1_uni_s['pageViewsMin'] = tp1_uni_s['pageviews']
tp1_uni_s['pageViewsMax'] = tp1_uni_s['pageviews']
tp1_uni_s['pageViewsMean'] = tp1_uni_s['pageviews']
tp1_uni_s['pageViewsStd'] = 0

In [26]:
tp1_uni_s.drop(columns = ['date','hits','pageviews'], inplace=True)

In [27]:
tp1_ = pd.concat([tp1_dup_s, tp1_uni_s])

In [28]:
# TP2 Process
groupSize2 = tp2.groupby('fullVisitorId').size().to_frame()
groupSize2.columns=['size']
tp2size = tp2.merge(groupSize2,how='left',left_on='fullVisitorId',right_index=True)
tp2_dup = tp2size[tp2size['size']>1]
tp2_uni = tp2size[tp2size['size']==1]

In [29]:
tp2_dup_s = tp2_dup.groupby('fullVisitorId').agg({'bounces':'sum',\
                                  'browser':lambda x:x.value_counts().index[0],\
                                  'campaign':lambda x:x.value_counts().index[0],\
                                  'channelGrouping':'max',\
                                  'city':lambda x:x.value_counts().index[0],\
                                  'continent':lambda x:x.value_counts().index[0],\
                                  'country':lambda x:x.value_counts().index[0],\
                                  'date':['count',lambda x: (x.max() - x.min()).days],\
                                  'deviceCategory':lambda x:x.value_counts().index[0],\
                                  'hits':['sum','min','max','mean','std'],\
                                  'isMobile':lambda x:x.value_counts().index[0],\
                                  'isTrueDirect':lambda x:x.value_counts().index[0],\
                                  'medium':lambda x:x.value_counts().index[0],\
                                  'metro':lambda x:x.value_counts().index[0],\
                                  'networkDomain':lambda x:x.value_counts().index[0],\
                                  'newVisits':'max',
                                  'operatingSystem':lambda x:x.value_counts().index[0],\
                                  'pageviews':['sum','min','max','mean','std'],\
                                  'region':lambda x:x.value_counts().index[0],\
                                  'source':lambda x:x.value_counts().index[0],\
                                  'subContinent':lambda x:x.value_counts().index[0],\
                                  'timeOnSite':'sum',\
                                  'transactionRevenue':'sum',\
                                  'transactions':'max',\
                                  'visitNumber':'max'})


In [30]:
tp2_dup_s.columns = ['bounces','browser','campaign','channelGrouping','city',\
                            'continent','country','dateCount','dateDiff','deviceCategory','hitsSum',\
                            'hitsMin','hitsMax','hitsMean','hitsStd','isMobile','isTrueDirect','medium',\
                            'metro','networkDomain','newVisits','operatingSystem','pageViewsSum','pageViewsMin',\
                           'pageViewsMax','pageViewsMean','pageViewsStd','region','source','subContinent',\
                            'timeOnSite','transactionRevenue','transactions','visitNumber']

In [31]:
tp2_uni_s = tp2_uni[['bounces','browser','campaign','channelGrouping','city',\
                    'continent','country','date','deviceCategory','hits',\
                    'isMobile','isTrueDirect','medium',\
                    'metro','networkDomain','newVisits','operatingSystem','pageviews',\
                    'region','source','subContinent',\
                    'timeOnSite','transactionRevenue','transactions','visitNumber']]

In [32]:
tp2_uni_s['dateCount'] = 1
tp2_uni_s['dateDiff'] = 0
tp2_uni_s['hitsSum'] = tp2_uni_s['hits']
tp2_uni_s['hitsMin'] = tp2_uni_s['hits']
tp2_uni_s['hitsMax'] = tp2_uni_s['hits']
tp2_uni_s['hitsMean'] = tp2_uni_s['hits']
tp2_uni_s['hitsStd'] = 0
tp2_uni_s['pageViewsSum'] = tp2_uni_s['pageviews']
tp2_uni_s['pageViewsMin'] = tp2_uni_s['pageviews']
tp2_uni_s['pageViewsMax'] = tp2_uni_s['pageviews']
tp2_uni_s['pageViewsMean'] = tp2_uni_s['pageviews']
tp2_uni_s['pageViewsStd'] = 0

In [33]:
tp2_uni_s.drop(columns = ['date','hits','pageviews'], inplace=True)

In [34]:
tp2_ = pd.concat([tp2_dup_s, tp2_uni_s])

In [35]:
#TP3

In [36]:
groupSize3 = tp3.groupby('fullVisitorId').size().to_frame()
groupSize3.columns=['size']
tp3size = tp3.merge(groupSize1,how='left',left_on='fullVisitorId',right_index=True)
tp3_dup = tp3size[tp3size['size']>1]
tp3_uni = tp3size[tp3size['size']==1]

In [37]:
tp3_dup_s = tp3_dup.groupby('fullVisitorId').agg({'bounces':'sum',\
                                  'browser':lambda x:x.value_counts().index[0],\
                                  'campaign':lambda x:x.value_counts().index[0],\
                                  'channelGrouping':'max',\
                                  'city':lambda x:x.value_counts().index[0],\
                                  'continent':lambda x:x.value_counts().index[0],\
                                  'country':lambda x:x.value_counts().index[0],\
                                  'date':['count',lambda x: (x.max() - x.min()).days],\
                                  'deviceCategory':lambda x:x.value_counts().index[0],\
                                  'hits':['sum','min','max','mean','std'],\
                                  'isMobile':lambda x:x.value_counts().index[0],\
                                  'isTrueDirect':lambda x:x.value_counts().index[0],\
                                  'medium':lambda x:x.value_counts().index[0],\
                                  'metro':lambda x:x.value_counts().index[0],\
                                  'networkDomain':lambda x:x.value_counts().index[0],\
                                  'newVisits':'max',
                                  'operatingSystem':lambda x:x.value_counts().index[0],\
                                  'pageviews':['sum','min','max','mean','std'],\
                                  'region':lambda x:x.value_counts().index[0],\
                                  'source':lambda x:x.value_counts().index[0],\
                                  'subContinent':lambda x:x.value_counts().index[0],\
                                  'timeOnSite':'sum',\
                                  'transactionRevenue':'sum',\
                                  'transactions':'max',\
                                  'visitNumber':'max'})


In [38]:
tp3_dup_s.columns = ['bounces','browser','campaign','channelGrouping','city',\
                            'continent','country','dateCount','dateDiff','deviceCategory','hitsSum',\
                            'hitsMin','hitsMax','hitsMean','hitsStd','isMobile','isTrueDirect','medium',\
                            'metro','networkDomain','newVisits','operatingSystem','pageViewsSum','pageViewsMin',\
                           'pageViewsMax','pageViewsMean','pageViewsStd','region','source','subContinent',\
                            'timeOnSite','transactionRevenue','transactions','visitNumber']
tp3_uni_s = tp3_uni[['bounces','browser','campaign','channelGrouping','city',\
                    'continent','country','date','deviceCategory','hits',\
                    'isMobile','isTrueDirect','medium',\
                    'metro','networkDomain','newVisits','operatingSystem','pageviews',\
                    'region','source','subContinent',\
                    'timeOnSite','transactionRevenue','transactions','visitNumber']]

tp3_uni_s['dateCount'] = 1
tp3_uni_s['dateDiff'] = 0
tp3_uni_s['hitsSum'] = tp3_uni_s['hits']
tp3_uni_s['hitsMin'] = tp3_uni_s['hits']
tp3_uni_s['hitsMax'] = tp3_uni_s['hits']
tp3_uni_s['hitsMean'] = tp3_uni_s['hits']
tp3_uni_s['hitsStd'] = 0
tp3_uni_s['pageViewsSum'] = tp3_uni_s['pageviews']
tp3_uni_s['pageViewsMin'] = tp3_uni_s['pageviews']
tp3_uni_s['pageViewsMax'] = tp3_uni_s['pageviews']
tp3_uni_s['pageViewsMean'] = tp3_uni_s['pageviews']
tp3_uni_s['pageViewsStd'] = 0

tp3_uni_s.drop(columns = ['date','hits','pageviews'], inplace=True)
tp3_ = pd.concat([tp3_dup_s, tp3_uni_s])

In [39]:
# TP4

In [40]:
groupSize4 = tp4.groupby('fullVisitorId').size().to_frame()
groupSize4.columns=['size']
tp4size = tp4.merge(groupSize4,how='left',left_on='fullVisitorId',right_index=True)
tp4_dup = tp4size[tp4size['size']>1]
tp4_uni = tp4size[tp4size['size']==1]

In [41]:
tp4_dup_s = tp4_dup.groupby('fullVisitorId').agg({'bounces':'sum',\
                                  'browser':lambda x:x.value_counts().index[0],\
                                  'campaign':lambda x:x.value_counts().index[0],\
                                  'channelGrouping':'max',\
                                  'city':lambda x:x.value_counts().index[0],\
                                  'continent':lambda x:x.value_counts().index[0],\
                                  'country':lambda x:x.value_counts().index[0],\
                                  'date':['count',lambda x: (x.max() - x.min()).days],\
                                  'deviceCategory':lambda x:x.value_counts().index[0],\
                                  'hits':['sum','min','max','mean','std'],\
                                  'isMobile':lambda x:x.value_counts().index[0],\
                                  'isTrueDirect':lambda x:x.value_counts().index[0],\
                                  'medium':lambda x:x.value_counts().index[0],\
                                  'metro':lambda x:x.value_counts().index[0],\
                                  'networkDomain':lambda x:x.value_counts().index[0],\
                                  'newVisits':'max',
                                  'operatingSystem':lambda x:x.value_counts().index[0],\
                                  'pageviews':['sum','min','max','mean','std'],\
                                  'region':lambda x:x.value_counts().index[0],\
                                  'source':lambda x:x.value_counts().index[0],\
                                  'subContinent':lambda x:x.value_counts().index[0],\
                                  'timeOnSite':'sum',\
                                  'transactionRevenue':'sum',\
                                  'transactions':'max',\
                                  'visitNumber':'max'})

In [42]:
tp4_dup_s.columns = ['bounces','browser','campaign','channelGrouping','city',\
                            'continent','country','dateCount','dateDiff','deviceCategory','hitsSum',\
                            'hitsMin','hitsMax','hitsMean','hitsStd','isMobile','isTrueDirect','medium',\
                            'metro','networkDomain','newVisits','operatingSystem','pageViewsSum','pageViewsMin',\
                           'pageViewsMax','pageViewsMean','pageViewsStd','region','source','subContinent',\
                            'timeOnSite','transactionRevenue','transactions','visitNumber']
tp4_uni_s = tp4_uni[['bounces','browser','campaign','channelGrouping','city',\
                    'continent','country','date','deviceCategory','hits',\
                    'isMobile','isTrueDirect','medium',\
                    'metro','networkDomain','newVisits','operatingSystem','pageviews',\
                    'region','source','subContinent',\
                    'timeOnSite','transactionRevenue','transactions','visitNumber']]

tp4_uni_s['dateCount'] = 1
tp4_uni_s['dateDiff'] = 0
tp4_uni_s['hitsSum'] = tp4_uni_s['hits']
tp4_uni_s['hitsMin'] = tp4_uni_s['hits']
tp4_uni_s['hitsMax'] = tp4_uni_s['hits']
tp4_uni_s['hitsMean'] = tp4_uni_s['hits']
tp4_uni_s['hitsStd'] = 0
tp4_uni_s['pageViewsSum'] = tp4_uni_s['pageviews']
tp4_uni_s['pageViewsMin'] = tp4_uni_s['pageviews']
tp4_uni_s['pageViewsMax'] = tp4_uni_s['pageviews']
tp4_uni_s['pageViewsMean'] = tp4_uni_s['pageviews']
tp4_uni_s['pageViewsStd'] = 0

tp4_uni_s.drop(columns = ['date','hits','pageviews'], inplace=True)
tp4_ = pd.concat([tp4_dup_s, tp4_uni_s])

In [43]:
# target Y: log(∑transactionRevenue+1)
Y_rw1 = rw1.groupby('fullVisitorId').transactionRevenue.sum().apply(lambda x: math.log(x+1)).to_frame()
Y_rw2 = rw2.groupby('fullVisitorId').transactionRevenue.sum().apply(lambda x: math.log(x+1)).to_frame()
Y_rw3 = rw3.groupby('fullVisitorId').transactionRevenue.sum().apply(lambda x: math.log(x+1)).to_frame()
Y_rw4 = rw4.groupby('fullVisitorId').transactionRevenue.sum().apply(lambda x: math.log(x+1)).to_frame()

In [44]:
tp1_.merge(Y_rw1,how = 'left', left_index= True, right_on = 'fullVisitorId').transactionRevenue_y.isna().sum()

387078

In [45]:
modelData1 = tp1_.merge(Y_rw1,how = 'left', left_index= True, right_on = 'fullVisitorId')
modelData2 = tp2_.merge(Y_rw2,how = 'left', left_index= True, right_on = 'fullVisitorId')
modelData3 = tp3_.merge(Y_rw3,how = 'left', left_index= True, right_on = 'fullVisitorId')
modelData4 = tp4_.merge(Y_rw4,how = 'left', left_index= True, right_on = 'fullVisitorId')

In [46]:
modelData1.transactionRevenue_y.fillna(0,inplace = True)
modelData2.transactionRevenue_y.fillna(0,inplace = True)
modelData3.transactionRevenue_y.fillna(0,inplace = True)
modelData4.transactionRevenue_y.fillna(0,inplace = True)

In [47]:
# The first 3 time periods is used as training dataset, the last one is used for testing
model_train = pd.concat([modelData1, modelData2, modelData3])
model_test = modelData4

In [48]:
#model_train.browser.nunique()
#model_train.campaign.nunique()

#model_train.channelGrouping.nunique()
#model_train.continent.nunique()
#model_train.deviceCategory.nunique()
#model_train.medium.nunique()
#model_train.operatingSystem.nunique()
#model_train.subContinent.nunique()

In [49]:
dropcol2 = ['browser','campaign','city','country','metro','networkDomain','region','source']
model_train.drop(columns = dropcol2, inplace=True)

In [50]:
model_test.drop(columns = dropcol2, inplace=True)

In [51]:
# label encoding the categorical variables
cat_f = [
    _f for _f in model_train.columns
    if model_train[_f].dtype == 'object'
]

In [52]:
for f in cat_f:
    model_train[f], indexer = pd.factorize(model_train[f])
    model_test[f] = indexer.get_indexer(model_test[f])

In [53]:
model_train_X = model_train.drop(['transactionRevenue_y'], axis=1)
model_train_Y = model_train.transactionRevenue_y.fillna(0)
model_test_X = model_test.drop(['transactionRevenue_y'], axis=1)
model_test_Y = model_test.transactionRevenue_y.fillna(0)

In [54]:
# generate a binary version of the target to denote if the customer returns or not
model_train_Y_bin = model_train_Y.apply(lambda x: 1 if x>0 else 0)
model_test_Y_bin = model_test_Y.apply(lambda x: 1 if x>0 else 0)

In [55]:
# run the first LightGBM model to get the return label
lgb_model_1 = lgb.LGBMClassifier(
        num_leaves=50,
        learning_rate=0.01,
        max_depth=6,
        n_estimators=1000,
        subsample=.9,
        colsample_bytree=.9,
        random_state=1)
lgb_model_1.fit(
        model_train_X, model_train_Y_bin,
        eval_set=[(model_test_X, model_test_Y_bin)],
        early_stopping_rounds=100,
        verbose=100,
        eval_metric='binary_logloss')

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.00120325	valid_0's binary_logloss: 0.00120325
[200]	valid_0's binary_logloss: 0.00114542	valid_0's binary_logloss: 0.00114542
[300]	valid_0's binary_logloss: 0.00112698	valid_0's binary_logloss: 0.00112698
[400]	valid_0's binary_logloss: 0.00112407	valid_0's binary_logloss: 0.00112407
Early stopping, best iteration is:
[390]	valid_0's binary_logloss: 0.00112388	valid_0's binary_logloss: 0.00112388


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
        importance_type='split', learning_rate=0.01, max_depth=6,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=1000, n_jobs=-1, num_leaves=50, objective=None,
        random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.9, subsample_for_bin=200000, subsample_freq=0)

In [56]:
model_train_return = model_train[model_train.transactionRevenue_y>0]
model_train_return_X = model_train_return.drop(['transactionRevenue_y'], axis=1)
model_train_return_Y = model_train_return.transactionRevenue_y.fillna(0)

model_test_return = model_test[model_test.transactionRevenue_y>0]
model_test_return_X = model_test_return.drop(['transactionRevenue_y'], axis=1)
model_test_return_Y = model_test_return.transactionRevenue_y.fillna(0)

In [57]:
model_train_return.shape
model_train.shape
model_test_return.shape
model_test.shape

(157, 27)

(685628, 27)

(56, 27)

(379358, 27)

In [58]:
# Run the second LightGBM model on the returned customers
lgb_model_2 = lgb.LGBMRegressor(
        num_leaves=30,
        learning_rate=0.001,
        max_depth=-1,
        n_estimators=1000,
        subsample=.9,
        colsample_bytree=.9,
        random_state=1)
lgb_model_2.fit(
        model_train_return_X, model_train_return_Y,
        eval_set=[(model_test_return_X, model_test_return_Y)],
        early_stopping_rounds=100,
        verbose=100,
        eval_metric='rmse')

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.27277	valid_0's l2: 1.61994
[200]	valid_0's rmse: 1.24734	valid_0's l2: 1.55585
[300]	valid_0's rmse: 1.22863	valid_0's l2: 1.50954
[400]	valid_0's rmse: 1.21432	valid_0's l2: 1.47456
[500]	valid_0's rmse: 1.20487	valid_0's l2: 1.45172
[600]	valid_0's rmse: 1.19946	valid_0's l2: 1.43872
[700]	valid_0's rmse: 1.19669	valid_0's l2: 1.43206
[800]	valid_0's rmse: 1.19607	valid_0's l2: 1.43058
Early stopping, best iteration is:
[775]	valid_0's rmse: 1.19575	valid_0's l2: 1.42981


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
       importance_type='split', learning_rate=0.001, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=1000, n_jobs=-1, num_leaves=30, objective=None,
       random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=0.9, subsample_for_bin=200000, subsample_freq=0)

We get a RMSE of 1.4

In [59]:
# process the dataset for predicting

groupSize = test.groupby('fullVisitorId').size().to_frame()
groupSize.columns=['size']
testsize = test.merge(groupSize,how='left',left_on='fullVisitorId',right_index=True)
test_dup = testsize[testsize['size']>1]
test_uni = testsize[testsize['size']==1]

In [60]:
test_dup_s = test_dup.groupby('fullVisitorId').agg({'bounces':'sum',\
                                  'browser':lambda x:x.value_counts().index[0],\
                                  'campaign':lambda x:x.value_counts().index[0],\
                                  'channelGrouping':'max',\
                                  'city':lambda x:x.value_counts().index[0],\
                                  'continent':lambda x:x.value_counts().index[0],\
                                  'country':lambda x:x.value_counts().index[0],\
                                  'date':['count',lambda x: (x.max() - x.min()).days],\
                                  'deviceCategory':lambda x:x.value_counts().index[0],\
                                  'hits':['sum','min','max','mean','std'],\
                                  'isMobile':lambda x:x.value_counts().index[0],\
                                  'isTrueDirect':'max',\
                                  'medium':lambda x:x.value_counts().index[0],\
                                  'metro':lambda x:x.value_counts().index[0],\
                                  'networkDomain':lambda x:x.value_counts().index[0],\
                                  'newVisits':'max',
                                  'operatingSystem':lambda x:x.value_counts().index[0],\
                                  'pageviews':['sum','min','max','mean','std'],\
                                  'region':lambda x:x.value_counts().index[0],\
                                  'source':lambda x:x.value_counts().index[0],\
                                  'subContinent':lambda x:x.value_counts().index[0],\
                                  'timeOnSite':'sum',\
                                  'transactionRevenue':'sum',\
                                  'transactions':'max',\
                                  'visitNumber':'max'})

In [61]:
test_dup_s.columns = ['bounces','browser','campaign','channelGrouping','city',\
                            'continent','country','dateCount','dateDiff','deviceCategory','hitsSum',\
                            'hitsMin','hitsMax','hitsMean','hitsStd','isMobile','isTrueDirect','medium',\
                            'metro','networkDomain','newVisits','operatingSystem','pageViewsSum','pageViewsMin',\
                           'pageViewsMax','pageViewsMean','pageViewsStd','region','source','subContinent',\
                            'timeOnSite','transactionRevenue','transactions','visitNumber']
test_uni_s = test_uni[['bounces','browser','campaign','channelGrouping','city',\
                    'continent','country','date','deviceCategory','hits',\
                    'isMobile','isTrueDirect','medium',\
                    'metro','networkDomain','newVisits','operatingSystem','pageviews',\
                    'region','source','subContinent',\
                    'timeOnSite','transactionRevenue','transactions','visitNumber']]

test_uni_s['dateCount'] = 1
test_uni_s['dateDiff'] = 0
test_uni_s['hitsSum'] = test_uni_s['hits']
test_uni_s['hitsMin'] = test_uni_s['hits']
test_uni_s['hitsMax'] = test_uni_s['hits']
test_uni_s['hitsMean'] = test_uni_s['hits']
test_uni_s['hitsStd'] = 0
test_uni_s['pageViewsSum'] = test_uni_s['pageviews']
test_uni_s['pageViewsMin'] = test_uni_s['pageviews']
test_uni_s['pageViewsMax'] = test_uni_s['pageviews']
test_uni_s['pageViewsMean'] = test_uni_s['pageviews']
test_uni_s['pageViewsStd'] = 0

test_uni_s.drop(columns = ['date','hits','pageviews'], inplace=True)
test_ = pd.concat([test_dup_s, test_uni_s])

test_.drop(columns = dropcol2, inplace=True)

for f in cat_f:
    test_[f], indexer = pd.factorize(test_[f])

In [62]:
test_.rename(columns={"transactionRevenue": "transactionRevenue_x"},inplace=True)

In [63]:
test_.isTrueDirect.fillna(False , inplace=True)

In [64]:
# Predict
predict_returnLabel = lgb_model_1.predict(test_)
predict_targerValue = lgb_model_2.predict(test_)

In [65]:
returnLabel = pd.Series(predict_returnLabel, index = test_.index).\
to_frame().reset_index()
targetValue = pd.Series(predict_targerValue, index = test_.index).\
to_frame().reset_index()

In [66]:
result = returnLabel.merge(targetValue, left_on='index',right_on='index')

In [67]:
result.columns = ['fullVisitorId','returnLabel', 'targetValue']

In [68]:
# if returnLabel is 0 then we take 0, 
# if returnLabel is 1 then we take the value given by the second LGB regression model
result['PredictedLogRevenue'] = result.returnLabel*result.targetValue

In [69]:
result.drop(['returnLabel','targetValue'], axis=1,inplace=True)
result.head()

,fullVisitorId,PredictedLogRevenue
0,82829325851015,0.0
1,121818656106736,0.0
2,143452507824332,0.0
3,187558509199268,0.0
4,217190245033763,0.0


In [70]:
result.to_csv('result.csv',index=False)